#Kaggle titanic

In [1]:
import graphlab

A newer version of GraphLab Create (v1.7.1) is available! Your current version is v1.6.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


##Load the data

In [2]:
passengers = graphlab.SFrame('train.csv')

[INFO] This non-commercial license of GraphLab Create is assigned to vishnu.viswanath25@gmail.com and will expire on November 13, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-2861 - Server binary: /Users/vishnu/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1448941811.log
[INFO] GraphLab Server Version: 1.6.1


PROGRESS: Finished parsing file /Users/vishnu/git/hadoop/ipython/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.030947 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/vishnu/git/hadoop/ipython/train.csv
PROGRESS: Parsing completed. Parsed 891 lines in 0.012727 secs.


##Analyze

In [3]:
graphlab.canvas.set_target('ipynb')
passengers.show()

##Pre process
Age column has null values, fill it with Avg age

In [4]:
passengers = passengers.fillna("Age",passengers["Age"].mean())

##Feature selection

In [5]:
features = ["Pclass","Sex","Age"]

##Model building
Split data into train and test set

In [6]:
train,test = passengers.random_split(0.8,seed=0)

In [7]:
model = graphlab.logistic_classifier.create(train,
                                            target="Survived",
                                            features = features,
                                            validation_set = test)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 703
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: Number of coefficients    : 4
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 1.002525     | 0.802276          | 0.734043            |
PROGRESS: | 2         | 3        | 1.004215     | 0.812233          | 0.750000            |
PROGRESS: | 3         | 4        | 1.005811     | 0.805121          | 0.707447            |
PROGRESS: | 4         | 5        | 1.007

##Evaluation
ROC curve

In [8]:
model.evaluate(test,metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-----+-----+----+-----+
 |    threshold     | fpr | tpr | p  |  n  |
 +------------------+-----+-----+----+-----+
 |       0.0        | 0.0 | 0.0 | 73 | 114 |
 | 0.0010000000475  | 1.0 | 1.0 | 73 | 114 |
 | 0.00200000009499 | 1.0 | 1.0 | 73 | 114 |
 | 0.00300000002608 | 1.0 | 1.0 | 73 | 114 |
 | 0.00400000018999 | 1.0 | 1.0 | 73 | 114 |
 | 0.00499999988824 | 1.0 | 1.0 | 73 | 114 |
 | 0.00600000005215 | 1.0 | 1.0 | 73 | 114 |
 | 0.00700000021607 | 1.0 | 1.0 | 73 | 114 |
 | 0.00800000037998 | 1.0 | 1.0 | 73 | 114 |
 | 0.00899999961257 | 1.0 | 1.0 | 73 | 114 |
 +------------------+-----+-----+----+-----+
 [1001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [9]:
model.show(view='Evaluation')

##Predict

In [10]:
passengers_submission = graphlab.SFrame('test.csv')

PROGRESS: Finished parsing file /Users/vishnu/git/hadoop/ipython/test.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.031113 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/vishnu/git/hadoop/ipython/test.csv
PROGRESS: Parsing completed. Parsed 418 lines in 0.011629 secs.


In [13]:
passengers_submission.show()

In [11]:
prediction = model.predict(passengers_submission,output_type='class')

In [12]:
prediction

dtype: int
Rows: 418
[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ... ]

In [15]:
passengers_submission["Survived"] = prediction

In [18]:
result = passengers_submission["PassengerId","Survived"]

In [19]:
result

PassengerId,Survived
892,0
893,0
894,0
895,0
896,1
897,0
898,1
899,0
900,1
901,0


In [20]:
result.save('submission.csv')